# CorDA

Revisión informal de "_CorDA: Context-Oriented Decomposition Adaptation of Large Language Models for Task-Aware Parameter-Efficient Fine-tuning_".

- [https://arxiv.org/pdf/2406.05223](https://arxiv.org/pdf/2406.05223)

Técnica publicada en junio de 2024 que propone un método alternativo para inicializar las matrices de bajo rango utilizadas en _LoRA_.

## 1. Arquitectura

_LoRA_ utiliza matrices de bajo rango para añadir nuevo conocimiento a un modelo preentrenado sin modificar los pesos aprendidos.

```
Weights x Input + (Weights' x Input) ─> Output
```

_CorDA_ propone inicializar las matrices de bajo rango a partir de las activaciones internas del modelo para un conjunto de muestras, y así orientar el reentrenamiento según el contexto.

## 2. Análisis

Revisa el reentrenamiento de modelos usando _LoRA_, argumenta que su inicialización aleatoria no considera ni el contexto de la nueva tarea ni la estructura de conocimiento del modelo preentrenado, y propone una alternativa.

### 2.1. Low-Rank Adaptation (LoRA)

_LoRA_ se basa en la premisa de que reentrenar un modelo con nuevo conocimiento requiere un cambio en los pesos que se encuentra en un subespacio de mucha menor dimensión que el espacio donde se encuentran los pesos aprendidos por el modelo.

- [https://github.com/jcmellado/sapiens/blob/main/notebooks/lora/lora.ipynb](https://github.com/jcmellado/sapiens/blob/main/notebooks/lora/lora.ipynb)

Lo que permite expresar dicho cambio de pesos como un producto de dos matrices de bajo rango.

Durante el reentrenamiento los pesos aprendidos del modelo se congelan, y se aprenden sólo las matrices de bajo rango, considerablemente más pequeñas.

- $ W^{*} = W + \Delta{W} = W + BA $

Siendo:

- $W^{*}$: los pesos después del reentrenamiento.

- $W$: los pesos originales del modelo, $W \in \mathbb{R}^{d_{out} \times d_{in}}$.

- $\Delta{W}$: el cambio de pesos debido al reentrenamiento.

- $BA$: la descomposición de $\Delta{W}$ en las matrices $B \in \mathbb{R}^{d_{out} \times r}$ y $A \in \mathbb{R}^{r \times d_{in}}$ con rango $r \ll \operatorname{min}(d_{out}, d_{in})$.

La matriz $A$ se inicializa habitualmente utilizando el método de _Kaiming_ (distribución normal de media cero y desviación típica $\sqrt{2 / d_{in}}$), y la matriz $B$ con ceros. Al ser cero la segunda matriz, el cambio de pesos inicial es cero, y el reentrenamiento empieza con los valores originales de los pesos aprendidos por el modelo.

La ventaja de esta técnica es que no requiere reentrenar el modelo completo, y el cambio de pesos aprendido se puede sumar directamente a los pesos del modelo durante la inferencia para añadir el nuevo conocimiento sin añadir latencia.

### 2.2. Covariance Matrix

_CorDA_ se apoya en la _matriz de covarianza_, por lo que es conveniente hacer una introducción informal a este concepto.

La _covarianza_ es un valor escalar que indica si dos variables tienden a variar en la misma dirección (covarianza positiva) o en direcciones opuestas (covarianza negativa).

Se calcula promediando la suma del producto de las desviaciones de cada variable respecto a su media.

- $ \operatorname{Cov}(X, Y) = \dfrac{1}{{n-1}} \sum\limits_{i=1}^n (x_i - \bar{x}) (y_i - \bar{y}) $

La _matriz de covarianza_ es una matriz cuadrada que contiene las covarianzas calculadas para todas las posibles combinaciones de pares de variables.

En la diagonal principal se sitúan las varianzas (covarianza de una variable consigo misma), y en el resto de posiciones las covarianzas entre las distintas combinaciones de variables.

- $ \begin{bmatrix}
\operatorname{Var}(X) & \operatorname{Cov}(X, Y)
\\ \operatorname{Cov}(Y, X) & \operatorname{Var}(Y)
\end{bmatrix}$

Este resultado se aplica a redes neuronales, por ejemplo para conocer la estructura de correlaciones en las activaciones de salida de una capa. Lo que se puede calcular a través de la matriz de covarianza $\Sigma_y$ de la salida $\mathbf{y}$, para una entrada $\mathbf{x}$ de media $\mu_x$ y matriz de covarianza $\Sigma_x$.

- $ \Sigma_y = \operatorname{Cov}(\mathbf{y}) = \operatorname{Cov}(\mathbf{W} \mathbf{x} + \mathbf{b}) = \mathbf{W} \Sigma_x \mathbf{W}^T $

La matriz $b$ de _bias_ es una constante y no influye, y el paso final se alcanza por un resultado conocido de la covarianza, si $\mathbf{y} = \mathbf{A} \mathbf{x}$, entonces $\operatorname{Cov}(\mathbf{y}) = \mathbf{A} \operatorname{Cov}(\mathbf{x}) \mathbf{A}^T$.

Es decir, que si se tiene la matriz de covarianza de la entrada, y los pesos, entonces se puede calcular la matriz de covarianza de la salida.

### 2.3. Inner Product

Aunque el _paper_ menciona la matriz de covarianza, lo que realmente se calcula es el _producto interno_ de una matriz por su traspuesta.

- $ X X^T $

Esta expresión no se corresponde exactamente con la expresión de cálculo de la covarianza, donde se restan las medias de los valores, pero es proporcional si se considera que los valores ya están centrados en una media $\mu$ próxima a cero.

- $ (X - \mu) (X - \mu)^T $

Lo único que falta es promediar, pero no hacerlo quiere decir simplemente que el resultado no está normalizado por $n - 1$.

La matriz resultante es similar a la de covarianza, pero no está centrada ni promediada.

Conserva las características internas relevantes y es computacionalmente más eficiente de calcular.

### 2.4. Singular Value Decomposition (SVD)

_CorDA_ se apoya en la _Descomposición en Valores Singulares_ (_SVD_), por lo que es conveniente realizar también una introducción informal a esta técnica.

_SVD_ descompone una matriz en tres matrices.

- $ A = U \Sigma V^T $

Siendo:

- $A \in \mathbb{R}^{m \times n}$: la matriz que se quiere descomponer.

- $U \in \mathbb{R}^{m \times m}$: una matriz ortogonal (formada por vectores unitarios y perpendiculares entre sí).

- $\Sigma \in \mathbb{R}^{m \times n}$: una matriz diagonal.

- $V \in \mathbb{R}^{n \times n}$: una matriz ortogonal.

Los vectores columnas de $U$ se llaman vectores singulares _izquierdos_, y los de $V$ se llaman vectores singulares _derechos_.

La matriz $\Sigma$ es una matriz diagonal que contiene los valores singulares de la matriz $A$ en orden descendente. Estos valores son las raíces cuadradas de los autovalores de $A^TA$. Siendo los _valores singulares_ un conjunto de escalares que indican como se escala un vector al multiplicarse por la matriz. Y los _autovalores_ escalares que miden cómo una matriz estira o comprime ciertos vectores. 

Si se considera que la matriz $A$ representa una transformación lineal aplicada sobre un conjunto de datos. La matriz $V^T$ gira los vectores originales de entrada, manteniendo los ángulos originales. La matriz $\Sigma$ escala los vectores, cambiando su magnitud. Y la matriz $U$ realiza otra rotación para llevar los vectores a su posición final, manteniendo aún los ángulos originales.

_SVD_ descompone una matriz en sus componentes principales, revelando la estructura interna de la matriz y las operaciones que realiza al aplicarla sobre un conjunto de datos.

### 2.5. Context-Oriented Decomposition

El argumento principal de _CorDA_ es que la inicialización aleatoria de _LoRA_ no tiene en cuenta ni el contexto de la nueva tarea ni la estructura de conocimiento del modelo. _CorDA_ propone inicializar las matrices de forma que se respeten los patrones de activación aprendidos por el modelo.

Toma un conjunto de muestras del _dataset_ de entrenamiento para la tarea específica que se quiere reentrenar (pares preguntas-respuestas, escritura de código, resolución de problemas matemáticos, etc). Realiza la inferencia con los pesos aprendidos del modelo, sin modificar. Y obtiene los valores de las activaciones a las entradas de las capas lineales del modelo.

A continuación calcula la matriz de covarianza de las activaciones en la entrada de cada capa lineal del modelo.

- $ C = X X^T \in \mathbb{R}^{d_{in} \times d_{in}}$

Y aplica _SVD_ al producto de la matriz de pesos de la capa lineal por la matriz de covarianza.

- $ \operatorname{SVD}(WC) = U \Sigma V^T $

Siendo:
 
- $W \in \mathbb{R}^{d_{out} \times d_{in}}$: la matriz de pesos de la capa lineal.

- $C \in \mathbb{R}^{d_{in} \times d_{in}}$: la matriz de covarianza.

- $U \in \mathbb{R}^{d_{out} \times d_{out}}$: la matriz ortogonal con los vectores singulares izquierdos. 

- $\Sigma \in \mathbb{R}^{d_{out} \times d_{in}}$: la matriz diagonal con los valores singulares en orden descendente en su diagonal principal.

- $V \in \mathbb{R}^{d_{in} \times d_{in}}$: la matriz ortogonal con los vectores singulares derechos.

Expresión que puede desarrollarse haciendo la multiplicación de las matrices.

- $ U \Sigma V^T = \displaystyle \sum\limits_{i=1}^R \sigma_i \mathbf{u}_i \mathbf{v}_i^T $

Siendo:

- $\sigma_i$: los valores singulares.

- $\mathbf{u}_i \in \mathbb{R}^{d_{out}}$: los vectores singulares izquierdos.

- $\mathbf{v}_i \in \mathbb{R}^{d_{in}}$: los vectores singulares derechos.

- $R \le \min(d_{out}, d_{in})$: el rango (número de valores singulares distintos de cero) de $WC$.

Calcular la matriz de covarianza sobre las activaciones de entrada significa obtener la correlación entre las distintas dimensiones de activación a lo largo de todo el conjunto de muestras. Es decir, la matriz $C$ captura cómo las diferentes características de entrada tienden a variar juntas en el contexto de una tarea.

Aplicar _SVD_ a los pesos multiplicados por la matriz de covarianza guía la descomposición de la matriz de pesos teniendo en cuenta la correlación entre las dimensiones de los _embeddings_. Orienta el proceso de factorización a través del contexto representativo de las muestras tomadas y la tarea específica realizada por el modelo para dichas muestras.

Los componentes $\mathbf{u}_i$ y $\mathbf{v}_i$ con los valores singulares $\sigma_i$ de mayor valor representan las características dominantes de la tarea asociada a $C$.

### 2.6. Fine-Tuning

_CorDA_ calcula unos nuevos pesos antes de realizar el _fine-tuning_ multiplicando el resultado de la descomposición con _SVD_ por la inversa de la matriz de covarianza.

- $ \hat{W} = \operatorname{SVD}(W C) C^{-1} = U \Sigma (V^T C^{-1}) = \displaystyle \sum\limits_{i=1}^{R} \sigma_i \mathbf{u}_i \mathbf{\hat{v}}_i^T $

Siendo:

- $\hat{W}$: los pesos inicializados para realizar el _fine-tuning_.

- $C^{-1}$: la inversa de la matriz de covarianza.

- $\mathbf{\hat{v}}_i^T$: el vector $i$-ésimo de la matriz $V^T C^{-1}$.

Si la matriz $C$ no es invertible, se aplica un proceso iterativo para hacerla invertible. Se suma a la diagonal de $C$ un valor positivo, calculado como un coeficiente multiplicado por la media de la diagonal. Y se repite la operación, doblando el coeficiente en cada paso, hasta que la distancia L2 entre $CC^{-1}$ y la matriz identidad es inferior a un umbral.

Esta inicialización preserva los valores originales de los pesos entrenados por el modelo al iniciar el reentrenamiento, de forma similar a lo que hace _LoRA_ inicializando con ceros.

Mientras que _LoRA_ cambia las capas para sumar a los pesos originales las matrices de bajo rango. Los pesos originales son congelados, y las matrices de bajo rango aprendidas.

- $ W^{*} = W + BA $

_CorDA_ calcula nuevos pesos a partir de los originales y las matrices de bajo rango inicializadas.

- $ W' = W - BA $

Y cambia las capas para utilizar los nuevos pesos y sumar las matrices de bajo rango. Los nuevos pesos son congelados, y las matrices de bajo rango aprendidas.

- $ W^{*} = W' + BA $

### 2.7. Knowledge-Preserved Adaptation

Para preservar el conocimiento general aprendido por un modelo durante el _fine-tuning_, _CorDA_ propone entrenar los últimos $r$ componentes con los valores singulares de menor valor, y congelar el resto de componentes.

- $ B = U_{[:,-r:]} \sqrt{\Sigma}_{[-r:]} $

- $ A = \sqrt{\Sigma}_{[-r:]} (V^T C^{-1})_{[-r:,:]} $

- $ BA = \displaystyle \sum\limits_{i=R-r+1}^R \sigma_i \mathbf{u}_i \mathbf{\hat{v}}_i^T $

Siendo $U_{[:,-r:]}$, $\sqrt{\Sigma}_{[-r:]}$, y $(V^T C^{-1})_{[-r:,:]}$, las últimas $r$ columnas de $U$, los últimos $r$ elementos de la diagonal de $\Sigma$, y las últimas $r$ filas de $V^T C^{-1}$, respectivamente.

La idea es que si se crea la matriz de covarianza con muestras de tipo pregunta-respuesta, los componentes con los valores singulares de mayor valor representan las características dominantes de este tipo de tareas donde se enfatiza el conocimiento general del modelo. Por lo tanto, entrenando los componentes con valores singulares de menor valor, y congelando los de mayor valor, se preserva el conocimiento general del modelo.

Congelar los de mayor valor quiere decir los primeros $R-r$ componentes de $W'$ se congelan y el resto se entrenan.

### 2.8. Instruction-Previewed Adaptation

Para favorecer el _fine-tuning_ de una tarea, _CorDA_ propone entrenar los primeros $r$ componentes con los valores singulares de mayor valor, y congelar el resto de componentes.

- $ B = U_{[:,:r]} \sqrt{\Sigma}_{[:r]} $

- $ A = \sqrt{\Sigma}_{[:r]} (V^T C^{-1})_{[:r,:]} $

- $ BA = \displaystyle \sum\limits_{i=1}^r \sigma_i \mathbf{u}_i \mathbf{\hat{v}}_i^T $

Siendo $U_{[:,:r]}$, $\sqrt{\Sigma}_{[:r]}$, y $(V^T C^{-1})_{[:r,:]}$, las primeras $r$ columnas de $U$, los primeros $r$ elementos de la diagonal de $\Sigma$, y las primeras $r$ filas de $V^T C^{-1}$, respectivamente.

La idea es que si se crea la matriz de covarianza con muestras de problemas de escritura de código, los componentes con los valores singulares de mayor valor representan las características dominantes de este tipo de tareas donde se enfatiza la capacidad del modelo de realizar esa tarea específica. Por lo tanto, entrenando los componentes con valores singulares de mayor valor, y congelando los de menor valor, se enfatiza el entrenamiento de la capacidad de realizar la tarea por parte del modelo.

Congelar los de menor valor quiere decir que los últimos $R-r$ componentes de $W'$ se congelan y el resto se entrenan.